# Conversión de Latitudes Anómalas a Sistemas de Coordenadas

No Funcionó

## Importar librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import utm

## Cargar base de datos

02desastres_definitivo.csv

In [2]:
df = pd.read_csv('../../Data/02ParaLimpiar/02desastres_paralimpiar.csv', delimiter=';', encoding='utf-8')
df.head(10)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
0,1900-9002-CPV,1900,9002,Climatological,Drought,Drought,Cabo Verde,CPV,Western Africa,Africa,...,NaN,1900,NaN,NaN,1900,NaN,NaN,11000.0,NaN,NaN
1,1900-9001-IND,1900,9001,Climatological,Drought,Drought,India,IND,Southern Asia,Asia,...,NaN,1900,NaN,NaN,1900,NaN,NaN,1250000.0,NaN,NaN
2,1902-0012-GTM,1902,12,Geophysical,Earthquake,Ground movement,Guatemala,GTM,Central America,Americas,...,-91,1902,4.0,18.0,1902,4.0,18.0,2000.0,NaN,843726.0
3,1902-0003-GTM,1902,3,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,4.0,8.0,1902,4.0,8.0,1000.0,NaN,NaN
4,1902-0010-GTM,1902,10,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,10.0,24.0,1902,10.0,24.0,6000.0,NaN,NaN
5,1903-0006-CAN,1903,6,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1903,4.0,29.0,1903,4.0,29.0,76.0,23.0,NaN
6,1903-0012-COM,1903,12,Geophysical,Volcanic activity,Ash fall,Comoros (the),COM,Eastern Africa,Africa,...,NaN,1903,NaN,NaN,1903,NaN,NaN,17.0,NaN,NaN
7,1904-0003-BGD,1904,3,Meteorological,Storm,Tropical cyclone,Bangladesh,BGD,Southern Asia,Asia,...,NaN,1904,11.0,NaN,1904,11.0,NaN,NaN,NaN,NaN
8,1905-0005-CAN,1905,5,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1905,8.0,13.0,1905,8.0,13.0,18.0,18.0,NaN
9,1905-0003-IND,1905,3,Geophysical,Earthquake,Ground movement,India,IND,Southern Asia,Asia,...,76.16,1905,4.0,4.0,1905,4.0,4.0,20000.0,NaN,812477.0


## Filtro Disaster Type == Flood

In [5]:
filter = df['Disaster Type'] == 'Flood'
df_flood = df[filter]

In [6]:
df_flood.head(2)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
12,1906-0023-BEL,1906,23,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,NaN,1906,5.0,14.0,1906,5.0,14.0,6.0,NaN,NaN
13,1906-0024-BEL,1906,24,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,NaN,1906,4.0,NaN,1906,4.0,NaN,NaN,NaN,NaN


In [7]:
df_flood.shape

(5808, 26)

In [8]:
df_flood['Disaster Type'].dtype

dtype('O')

# Limpieza de coordenadas

## Función para limpiar coordenadas filtrado para df_flood

In [9]:
import re
import numpy as np

# Convertir 'Latitude' y 'Longitude' a string
df['Latitude'] = df['Latitude'].astype(str)
df['Longitude'] = df['Longitude'].astype(str)

# Limpiar las coordinadas de latitude y longitude
df['Latitude'] = df['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x))
df['Longitude'] = df['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x))

# Quitar puntos finales
df['Latitude'] = df['Latitude'].str.rstrip('.')
df['Longitude'] = df['Longitude'].str.rstrip('.')

# # Reemplazar Norte y Este con valores positivos, Sur y Oeste con negativos
# df['Latitude'] = df['Latitude'].apply(lambda x: float(x) if x and x[-1] != 'S' else -float(x[:-1]) if x else np.nan)
# df['Longitude'] = df['Longitude'].apply(lambda x: float(x) if x and x[-1] != 'W' else -float(x[:-1]) if x else np.nan)

# Función para convertir latitude y longitude
def convert_coordinates(x):
    if not x:
        return np.nan
    
    # Remover puntos extras en decimales
    x = re.sub('\.(?=.*\.)', '', x)
    
    try:
        value = float(x)
        if x[-1] == 'S' or x[-1] == 'W':
            return -value
        else:
            return value
    except ValueError:
        return np.nan

# Convertir Latitude y Longitude a coordenadas
df['Latitude'] = df['Latitude'].apply(convert_coordinates)
df['Longitude'] = df['Longitude'].apply(convert_coordinates)

# Redondear decimales a 2 dígitos
df['Latitude'] = df['Latitude'].round(2)
df['Longitude'] = df['Longitude'].round(2)

# Identificar valores anómalos a partir de los grados,
# que no representen coordenadas: los que no estén dentro de 90 y -90 ni 180 y -180
anomalous_lat = ~df['Latitude'].between(-90, 90)
anomalous_lon = ~df['Longitude'].between(-180, 180)

# Configurar valores anómalos a NaN
# df.loc[anomalous_lat, 'Latitude'] = np.nan
# df.loc[anomalous_lon, 'Longitude'] = np.nan


In [21]:
# Crear listas vacías para almacenar valores anómalos de latitude y longitude
anomalous_lat = []
anomalous_lon = []

# Iterar sobre las filas e identificar valores anómalos
for index, row in df_flood.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Revisar si latitude y longitude son números válidos
    if pd.notnull(latitude) and pd.notnull(longitude):
        # Convertir latitude y longitude a tipo numérico si hay strings
        if isinstance(latitude, str):
            latitude = pd.to_numeric(latitude, errors='coerce')
        if isinstance(longitude, str):
            longitude = pd.to_numeric(longitude, errors='coerce')
        
        # Revisar si latitude está fuera de rango (-90 to 90)
        if latitude < -90 or latitude > 90:
            anomalous_lat.append(latitude)
        
        # Revisar si longitude está fuera de rango (-180 a 180)
        if longitude < -180 or longitude > 180:
            anomalous_lon.append(longitude)

# Mostrar valores anómalos de latitude y longitude
print("Anomalous Latitude Values:", anomalous_lat)
print("Anomalous Longitude Values:", anomalous_lon)


Anomalous Latitude Values: [564.778, -865.047, 270.633, 350.754, 388.497, 151.821, 940.044, 193.801, -295.025, -405.208, 125.947, 289.081, 213.521, 334.553, 420.681, 194.433, 301.135, 342.696, 370.528, -104.897, 465.994, -528.256, 398.677, 236.387, -189.538, 369.291, -344.809, 304.563, 359.161, 229.043, 491.358, 532.732, -617.945, 448.013, 439.958, 431.605, 165.919, 822.392, -140.095, 205.946, 240.587, -183.097, 246.174, 163.191, 396.151, 166.076, 759.777, 420.408, 183.122, -248.006, -202.305, -156.178, 358.444, 150.103, 287.404, -203.461, 500.043, 443.289, 136.883, 216.964, 439.958, 303.632, 139.188, 382.138, 388.278, 375.597, -164.638, -414.041, -383.652, 440.617, -881.664, -223.617, 270.326, 101.743, 420.683, -255.689, -121.145, -351.086, -115.042, -697.805, -441.033, 394.741, 738.005, 414.571, 666.576, 730.951, 484.049, -252.205, 440.617, 417.575, 335.131, 331.767, 262.074, 378.142, 380.626, -655.868, 107.121, 218.734, 268.946, 336.304, 194.811, 148.417, -398.648, 350.129, 343.432,

In [11]:
# # Convertir 'Latitude' y 'Longitude' a string
# df['Latitude'] = df['Latitude'].astype(str)
# df['Longitude'] = df['Longitude'].astype(str)

# # Limpiar las coordenadas latitude y longitude
# df['Latitude'] = df['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df['Longitude'] = df['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)

# # Eliminar puntos finales
# df['Latitude'] = df['Latitude'].str.rstrip('.')
# df['Longitude'] = df['Longitude'].str.rstrip('.')

# # Cambiar strings vacíos a NaN
# df['Latitude'] = df['Latitude'].replace('', np.nan)
# df['Longitude'] = df['Longitude'].replace('', np.nan)

# # Convertir coordenadas válidas a float
# df['Latitude'] = df['Latitude'].astype(float)
# df['Longitude'] = df['Longitude'].astype(float)

# # Identificar valores anómalos a partir de los grados
# anomalous_lat = df['Latitude'].abs() > 90
# anomalous_lon = df['Longitude'].abs() > 180

# # Configurar valores anómalos a NaN
# df.loc[anomalous_lat, 'Latitude'] = np.nan
# df.loc[anomalous_lon, 'Longitude'] = np.nan

In [12]:
# df_flood = df[(df['Disaster Subtype'] == 'Flood') & (df['Total Deaths'].notna())]

# # Limpiar coordenadas latitude y longitude para df_flood
# df_flood['Latitude'] = df_flood['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df_flood['Longitude'] = df_flood['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)

# # Borrar puntos finales
# df_flood['Latitude'] = df_flood['Latitude'].str.rstrip('.')
# df_flood['Longitude'] = df_flood['Longitude'].str.rstrip('.')

# # Cambiar strings vacíos a NaN
# df_flood['Latitude'] = df_flood['Latitude'].replace('', np.nan)
# df_flood['Longitude'] = df_flood['Longitude'].replace('', np.nan)

# # Convertir coordenadas válidas a float
# df_flood['Latitude'] = df_flood['Latitude'].astype(float)
# df_flood['Longitude'] = df_flood['Longitude'].astype(float)

# # Remover filas con coordenadas NaN
# df_flood = df_flood.dropna(subset=['Latitude', 'Longitude'])


In [13]:
# # Filtrar el df_flood DataFrame para Disaster Subtype 'Flood' con non-null values en columnas específicas
# df_flood = df[(df['Disaster Subtype'] == 'Flood') & (df['Total Affected'].notna()) & (df['Total Deaths'].notna()) & (df['Dis Mag Scale'].notna()) & (df['Dis Mag Value'].notna())]

# # Limpiar y convertir la columna 'Latitude'
# df_flood['Latitude'] = df_flood['Latitude'].astype(str).apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df_flood['Latitude'] = df_flood['Latitude'].str.rstrip('.')
# df_flood['Latitude'] = df_flood['Latitude'].replace('', np.nan)
# df_flood['Latitude'] = df_flood['Latitude'].astype(float)

# # Limpiar y convertir la columna 'Longitude'
# df_flood['Longitude'] = df_flood['Longitude'].astype(str).apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df_flood['Longitude'] = df_flood['Longitude'].str.rstrip('.')
# df_flood['Longitude'] = df_flood['Longitude'].replace('', np.nan)
# df_flood['Longitude'] = df_flood['Longitude'].astype(float)

# # Remover filas con NaN en coordenadas, 'Total Deaths', y 'Total Affected'
# df_flood = df_flood.dropna(subset=['Latitude', 'Longitude', 'Total Deaths', 'Total Affected'])


## Tests de verificación de limpieza de coordenadas

In [14]:
print(df_flood['Latitude'])
print(df_flood['Longitude'])

12       NaN
13       NaN
34       NaN
39       NaN
43       NaN
        ... 
16620    NaN
16628    NaN
16629    NaN
16631    NaN
16634    NaN
Name: Latitude, Length: 5808, dtype: object
12       NaN
13       NaN
34       NaN
39       NaN
43       NaN
        ... 
16620    NaN
16628    NaN
16629    NaN
16631    NaN
16634    NaN
Name: Longitude, Length: 5808, dtype: object


In [15]:
print(df_flood['Latitude'].dtype)
print(df_flood['Longitude'].dtype)

object
object


In [19]:
anomalous_latitudes = [564.778, -865.047, 270.633, 350.754, 388.497, 151.821, 940.044, 193.801, -295.025, -405.208, 125.947, 289.081, 213.521, 334.553, 420.681, 194.433, 301.135, 342.696, 370.528, -104.897, 465.994, -528.256, 398.677, 236.387, -189.538, 369.291, -344.809, 304.563, 359.161, 229.043, 491.358, 532.732, -617.945, 448.013, 439.958, 431.605, 165.919, 822.392, -140.095, 205.946, 240.587, -183.097, 246.174, 163.191, 396.151, 166.076, 759.777, 420.408, 183.122, -248.006, -202.305, -156.178, 358.444, 150.103, 287.404, -203.461, 500.043, 443.289, 136.883, 216.964, 439.958, 303.632, 139.188, 382.138, 388.278, 375.597, -164.638, -414.041, -383.652, 440.617, -881.664, -223.617, 270.326, 101.743, 420.683, -255.689, -121.145, -351.086, -115.042, -697.805, -441.033, 394.741, 738.005, 414.571, 666.576, 730.951, 484.049, -252.205, 440.617, 417.575, 335.131, 331.767, 262.074, 378.142, 380.626, -655.868, 107.121, 218.734, 268.946, 336.304, 194.811, 148.417, -398.648, 350.129, 343.432, 155.509, 468.147, -215.153, 440.617, 719.301, 208.137, 341.936, -506.017, -326.329]

anomalous_indices = []
for index, lat in enumerate(anomalous_latitudes):
    if lat < -90 or lat > 90:
        anomalous_indices.append(index)

print("Índices Anómalos de Latitud:", anomalous_indices)


Índices Anómalos de Latitud: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113]


## Intento fallido de convertir Latitudes Anómalas a Sistema de Coordenadas DMS

In [2]:
# La intención era cambiar los puntos para que, por ejemplo 564.778 se conviertiera en:
# 56º 47' y 78", sin embargo, el resultado es inválido
anomalous_latitudes = [564.778, -865.047, 270.633, 350.754, 388.497, 151.821, 940.044, 193.801, -295.025, -405.208, 125.947, 289.081, 213.521, 334.553, 420.681, 194.433, 301.135, 342.696, 370.528, -104.897, 465.994, -528.256, 398.677, 236.387, -189.538, 369.291, -344.809, 304.563, 359.161, 229.043, 491.358, 532.732, -617.945, 448.013, 439.958, 431.605, 165.919, 822.392, -140.095, 205.946, 240.587, -183.097, 246.174, 163.191, 396.151, 166.076, 759.777, 420.408, 183.122, -248.006, -202.305, -156.178, 358.444, 150.103, 287.404, -203.461, 500.043, 443.289, 136.883, 216.964, 439.958, 303.632, 139.188, 382.138, 388.278, 375.597, -164.638, -414.041, -383.652, 440.617, -881.664, -223.617, 270.326, 101.743, 420.683, -255.689, -121.145, -351.086, -115.042, -697.805, -441.033, 394.741, 738.005, 414.571, 666.576, 730.951, 484.049, -252.205, 440.617, 417.575, 335.131, 331.767, 262.074, 378.142, 380.626, -655.868, 107.121, 218.734, 268.946, 336.304, 194.811, 148.417, -398.648, 350.129, 343.432, 155.509, 468.147, -215.153, 440.617, 719.301, 208.137, 341.936, -506.017, -326.329]

def convert_latitude(lat):
    if lat > 90 or lat < -90:
        return "Coordenada Inválida"
    
    sign = "-" if lat < 0 else ""
    lat = abs(lat)
    
    degrees = int(lat)
    lat_minutes = (lat - degrees) * 60
    minutes = int(lat_minutes)
    lat_seconds = (lat_minutes - minutes) * 60
    seconds = round(lat_seconds, 2)
    
    return f"{sign}{degrees}° {minutes}' {seconds}\""

coherent_latitudes = [convert_latitude(lat) for lat in anomalous_latitudes]

# Mostrar las latitudes convertidas a sistema de coordenadas DMS
print(f"Latitudes Convertidas a Sistema de Coordenadas DMS")
for index, lat in enumerate(coherent_latitudes):
    print(f"Latitud Anómala: {anomalous_latitudes[index]} -> Coherent Latitude: {lat}")


Latitudes Convertidas a Sistema de Coordenadas DMS
Latitud Anómala: 564.778 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: -865.047 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 270.633 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 350.754 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 388.497 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 151.821 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 940.044 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 193.801 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: -295.025 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: -405.208 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 125.947 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 289.081 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 213.521 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 334.553 -> Coherent Latitude: Invalid Coordinate
Latitud Anómala: 420.6

## Intento fallido de convertir Latitudes Anómalas a Sistema de Coordenadas tipo Military Grid System MGRS

In [32]:
import utm

anomalous_latitudes = [564.778, -865.047, 270.633, 350.754, 388.497, 151.821, 940.044, 193.801, -295.025, -405.208, 125.947, 289.081, 213.521, 334.553, 420.681, 194.433, 301.135, 342.696, 370.528, -104.897, 465.994, -528.256, 398.677, 236.387, -189.538, 369.291, -344.809, 304.563, 359.161, 229.043, 491.358, 532.732, -617.945, 448.013, 439.958, 431.605, 165.919, 822.392, -140.095, 205.946, 240.587, -183.097, 246.174, 163.191, 396.151, 166.076, 759.777, 420.408, 183.122, -248.006, -202.305, -156.178, 358.444, 150.103, 287.404, -203.461, 500.043, 443.289, 136.883, 216.964, 439.958, 303.632, 139.188, 382.138, 388.278, 375.597, -164.638, -414.041, -383.652, 440.617, -881.664, -223.617, 270.326, 101.743, 420.683, -255.689, -121.145, -351.086, -115.042, -697.805, -441.033, 394.741, 738.005, 414.571, 666.576, 730.951, 484.049, -252.205, 440.617, 417.575, 335.131, 331.767, 262.074, 378.142, 380.626, -655.868, 107.121, 218.734, 268.946, 336.304, 194.811, 148.417, -398.648, 350.129, 343.432, 155.509, 468.147, -215.153, 440.617, 719.301, 208.137, 341.936, -506.017, -326.329]

def convert_to_mgrs(lat, lon):
    try:
        easting, northing, zone_number, zone_letter = utm.from_latlon(lat, lon)
        mgrs_coordinate = utm.to_mgrs(easting, northing, zone_number, northern=lat > 0)
        return mgrs_coordinate
    except:
        return "Coordenada Inválida"

mgrs_coordinates = [convert_to_mgrs(lat, 0.0) for lat in anomalous_latitudes]

# Mostrar las coordenadas convertidas a sistema MGRS
for index, lat in enumerate(anomalous_latitudes):
    print(f"Latitud Anómala: {lat} -> MGRS: {mgrs_coordinates[index]}")


Latitud Anómala: 564.778 -> MGRS: Coordenada Inválida
Latitud Anómala: -865.047 -> MGRS: Coordenada Inválida
Latitud Anómala: 270.633 -> MGRS: Coordenada Inválida
Latitud Anómala: 350.754 -> MGRS: Coordenada Inválida
Latitud Anómala: 388.497 -> MGRS: Coordenada Inválida
Latitud Anómala: 151.821 -> MGRS: Coordenada Inválida
Latitud Anómala: 940.044 -> MGRS: Coordenada Inválida
Latitud Anómala: 193.801 -> MGRS: Coordenada Inválida
Latitud Anómala: -295.025 -> MGRS: Coordenada Inválida
Latitud Anómala: -405.208 -> MGRS: Coordenada Inválida
Latitud Anómala: 125.947 -> MGRS: Coordenada Inválida
Latitud Anómala: 289.081 -> MGRS: Coordenada Inválida
Latitud Anómala: 213.521 -> MGRS: Coordenada Inválida
Latitud Anómala: 334.553 -> MGRS: Coordenada Inválida
Latitud Anómala: 420.681 -> MGRS: Coordenada Inválida
Latitud Anómala: 194.433 -> MGRS: Coordenada Inválida
Latitud Anómala: 301.135 -> MGRS: Coordenada Inválida
Latitud Anómala: 342.696 -> MGRS: Coordenada Inválida
Latitud Anómala: 370.528 

## Intento fallido de convertir Latitudes Anómalas a Sistema de Coordenadas tipo Universal Transverse Mercator (UTM)

In [33]:
import utm

anomalous_latitudes = [564.778, -865.047, 270.633, 350.754, 388.497, 151.821, 940.044, 193.801, -295.025, -405.208, 125.947, 289.081, 213.521, 334.553, 420.681, 194.433, 301.135, 342.696, 370.528, -104.897, 465.994, -528.256, 398.677, 236.387, -189.538, 369.291, -344.809, 304.563, 359.161, 229.043, 491.358, 532.732, -617.945, 448.013, 439.958, 431.605, 165.919, 822.392, -140.095, 205.946, 240.587, -183.097, 246.174, 163.191, 396.151, 166.076, 759.777, 420.408, 183.122, -248.006, -202.305, -156.178, 358.444, 150.103, 287.404, -203.461, 500.043, 443.289, 136.883, 216.964, 439.958, 303.632, 139.188, 382.138, 388.278, 375.597, -164.638, -414.041, -383.652, 440.617, -881.664, -223.617, 270.326, 101.743, 420.683, -255.689, -121.145, -351.086, -115.042, -697.805, -441.033, 394.741, 738.005, 414.571, 666.576, 730.951, 484.049, -252.205, 440.617, 417.575, 335.131, 331.767, 262.074, 378.142, 380.626, -655.868, 107.121, 218.734, 268.946, 336.304, 194.811, 148.417, -398.648, 350.129, 343.432, 155.509, 468.147, -215.153, 440.617, 719.301, 208.137, 341.936, -506.017, -326.329]

def convert_to_utm(lat, lon):
    try:
        easting, northing, zone_number, zone_letter = utm.from_latlon(lat, lon)
        return easting, northing, zone_number, zone_letter
    except:
        return None

utm_coordinates = [convert_to_utm(lat, 0.0) for lat in anomalous_latitudes]

# Mostrar coordenadas UTM
for index, lat in enumerate(anomalous_latitudes):
    utm_coordinate = utm_coordinates[index]
    if utm_coordinate is not None:
        easting, northing, zone_number, zone_letter = utm_coordinate
        print(f"Latitud Anómala: {lat} -> UTM: Este={easting}, Norte={northing}, Zone={zone_number}{zone_letter}")
    else:
        print(f"Latitud Anómala: {lat} -> Coordenada Inválida")


Anomalous Latitude: 564.778 -> Invalid Coordinate
Anomalous Latitude: -865.047 -> Invalid Coordinate
Anomalous Latitude: 270.633 -> Invalid Coordinate
Anomalous Latitude: 350.754 -> Invalid Coordinate
Anomalous Latitude: 388.497 -> Invalid Coordinate
Anomalous Latitude: 151.821 -> Invalid Coordinate
Anomalous Latitude: 940.044 -> Invalid Coordinate
Anomalous Latitude: 193.801 -> Invalid Coordinate
Anomalous Latitude: -295.025 -> Invalid Coordinate
Anomalous Latitude: -405.208 -> Invalid Coordinate
Anomalous Latitude: 125.947 -> Invalid Coordinate
Anomalous Latitude: 289.081 -> Invalid Coordinate
Anomalous Latitude: 213.521 -> Invalid Coordinate
Anomalous Latitude: 334.553 -> Invalid Coordinate
Anomalous Latitude: 420.681 -> Invalid Coordinate
Anomalous Latitude: 194.433 -> Invalid Coordinate
Anomalous Latitude: 301.135 -> Invalid Coordinate
Anomalous Latitude: 342.696 -> Invalid Coordinate
Anomalous Latitude: 370.528 -> Invalid Coordinate
Anomalous Latitude: -104.897 -> Invalid Coordin

In [20]:
anomalous_latitudes = [564.778, -865.047, 270.633, 350.754, 388.497, 151.821, 940.044, 193.801, -295.025, -405.208, 125.947, 289.081, 213.521, 334.553, 420.681, 194.433, 301.135, 342.696, 370.528, -104.897, 465.994, -528.256, 398.677, 236.387, -189.538, 369.291, -344.809, 304.563, 359.161, 229.043, 491.358, 532.732, -617.945, 448.013, 439.958, 431.605, 165.919, 822.392, -140.095, 205.946, 240.587, -183.097, 246.174, 163.191, 396.151, 166.076, 759.777, 420.408, 183.122, -248.006, -202.305, -156.178, 358.444, 150.103, 287.404, -203.461, 500.043, 443.289, 136.883, 216.964, 439.958, 303.632, 139.188, 382.138, 388.278, 375.597, -164.638, -414.041, -383.652, 440.617, -881.664, -223.617, 270.326, 101.743, 420.683, -255.689, -121.145, -351.086, -115.042, -697.805, -441.033, 394.741, 738.005, 414.571, 666.576, 730.951, 484.049, -252.205, 440.617, 417.575, 335.131, 331.767, 262.074, 378.142, 380.626, -655.868, 107.121, 218.734, 268.946, 336.304, 194.811, 148.417, -398.648, 350.129, 343.432, 155.509, 468.147, -215.153, 440.617, 719.301, 208.137, 341.936, -506.017, -326.329]

anomalous_indices = []
for index, lat in enumerate(anomalous_latitudes):
    if lat < -90 or lat > 90:
        anomalous_indices.append(index)

num_anomalous = len(anomalous_indices)

print("Número de Coordenadas Anómalas de Latitud:", num_anomalous)


Número de Coordenadas Anómalas de Latitud: 114


In [18]:
# Examinar filas específicas para verificar si el proceso de limpieza ha manejado los valores anómalos correctamente.
print(df_flood.loc[124, 'Latitude'])
print(df_flood.loc[124, 'Longitude'])

KeyError: 124